In [ ]:
!pip install transformers
!pip install transformers[sentencepiece]
!pip install datasets
#!pip install pythainlp
!!pip install sacrebleu

# Test Model

In [ ]:
import transformers
from transformers import MarianMTModel, MarianTokenizer

src_text = [
    ">>tha<< I am trying to set up high-level meetings in China to state our case and explain why we should be unblocked.",
]

model_name = "/content/drive/MyDrive/NMT Hackathon/model/Finetune_Final/checkpoint-346564"
tokenizer = MarianTokenizer.from_pretrained(model_name)
print(tokenizer.supported_language_codes)


['>>ewe<<', '>>sna<<', '>>lin<<', '>>toi_Latn<<', '>>ceb<<', '>>oss<<', '>>run<<', '>>mfe<<', '>>ilo<<', '>>zlm_Latn<<', '>>pes<<', '>>smo<<', '>>hil<<', '>>niu<<', '>>sag<<', '>>fij<<', '>>cmn_Hans<<', '>>nya<<', '>>tso<<', '>>war<<', '>>gil<<', '>>hau_Latn<<', '>>umb<<', '>>glv<<', '>>tvl<<', '>>ton<<', '>>zul<<', '>>kal<<', '>>pag<<', '>>cmn_Hant<<', '>>pus<<', '>>abk<<', '>>pap<<', '>>hat<<', '>>mkd<<', '>>tuk_Latn<<', '>>yor<<', '>>tuk<<', '>>sqi<<', '>>tir<<', '>>mlg<<', '>>tur<<', '>>ido_Latn<<', '>>mai<<', '>>ibo<<', '>>srp_Cyrl<<', '>>srp_Latn<<', '>>kir_Cyrl<<', '>>heb<<', '>>bos_Latn<<', '>>bak<<', '>>ast<<', '>>som<<', '>>tah<<', '>>chv<<', '>>kek_Latn<<', '>>lug<<', '>>vie<<', '>>wln<<', '>>isl<<', '>>hye<<', '>>mah<<', '>>yue_Hant<<', '>>crh_Latn<<', '>>amh<<', '>>nds<<', '>>pan_Guru<<', '>>xho<<', '>>ukr<<', '>>cat<<', '>>afr<<', '>>tat<<', '>>guj<<', '>>jpn<<', '>>mon<<', '>>eus<<', '>>nob<<', '>>glg<<', '>>ind<<', '>>sin<<', '>>cym<<', '>>zho_Hant<<', '>>zho_Hans<<', '

In [ ]:
model = MarianMTModel.from_pretrained(model_name)
translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True), num_beams = 5)

In [ ]:
translated

tensor([[64109, 13536,  4658, 18536, 18536,  1921,  5438,  7284,  3329,  3571,
          3556,  2242, 12324, 16823,  7933,  5003,  1921, 21900,  2488, 23376,
          3187,  9116,  2242, 17055, 16823, 23190, 14363,  5676,  9619,  1948,
         24190,  1807, 13627, 25757,  3995, 14905,  9760, 17115, 28306,  3613,
         15618,  3396,  2939, 13627,  9524,  9439,  9257,  1921, 17115,  3995,
          2670,  1807, 26029, 29242,  2488,  2711, 13908,  3613,  3995,     0]])

In [ ]:
tgt_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

In [ ]:
tgt_text 

['ฉันพยายามที่จะตั้งการประชุมระดับสูงในประเทศจีนเพื่อรายงานคดีของเราและอธิบายว่าทําไมเราควรถูกปลดล็อค']

# Preparing the data

In [ ]:
from datasets import load_dataset, load_metric , concatenate_datasets , interleave_datasets

In [ ]:
!gdown --id 1XIfmTam2urnFoRvVLPEfo-fsrvluGjuF

Downloading...
From: https://drive.google.com/uc?id=1XIfmTam2urnFoRvVLPEfo-fsrvluGjuF
To: /content/googletrans_dataset.csv
100% 102M/102M [00:00<00:00, 256MB/s] 


In [ ]:
from datasets import load_dataset, load_metric , concatenate_datasets , interleave_datasets
raw_datasets_opus = load_dataset("opus100", "en-th")
raw_datasets_scb = load_dataset("scb_mt_enth_2020","enth")

Downloading:   0%|          | 0.00/2.29k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.3k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/38.1M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset scb_mt_enth2020 downloaded and prepared to /root/.cache/huggingface/datasets/scb_mt_enth2020/enth/1.0.0/3bb5a43042a84446b254e215a8b1604b454ad56ada3370309a6f6a3b480c86a8. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
import pandas as pd

google_df = pd.read_csv('/content/googletrans_dataset.csv')

In [ ]:
google_df.sample(10)

,en,th
127042,A database shall provide a facility that allow...,ฐานข้อมูลต้องจัดให้มีสิ่งอำนวยความสะดวกที่อนุญ...
2356,"Where such prices are not available, the compe...",หากไม่มีราคาดังกล่าว เจ้าหน้าที่ผู้มีอำนาจจะต้...
48747,It is neither to prohibit imports nor to hampe...,ไม่เป็นการห้ามการนำเข้าหรือขัดขวางกิจกรรมของผู...
112705,It shall also include the powers to take furth...,นอกจากนี้ยังรวมถึงอำนาจในการตัดสินใจเพิ่มเติมเ...
108308,"Detection, diagnosis and monitoring: to develo...",การตรวจจับ การวินิจฉัย และการเฝ้าติดตาม: เพื่อ...
79927,"According to its conclusions, strong economic ...",จากข้อสรุปดังกล่าว ประสิทธิภาพทางเศรษฐกิจที่แข...
47185,"For the remaining three companies, the Commiss...",สำหรับบริษัทที่เหลืออีกสามบริษัท คณะกรรมาธิการ...
58862,As the STECF estimate of the 2004 year-class o...,เนื่องจาก STECF ประมาณการของ Sandeel North Sea...
37757,"For the purpose of this Article, ‘common platf...",สำหรับวัตถุประสงค์ของบทความนี้ 'แพลตฟอร์มทั่วไ...
260,"OJ L 366, 24.12.1987, p. 1. Regulation as last...","OJ L 366, 24.12.1987, หน้า 1. ระเบียบที่แก้ไขล..."


In [ ]:
raw_datasets_google = load_dataset("csv", data_files="/content/googletrans_dataset.csv")

Using custom data configuration default-8408644d0de3a5a1


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-8408644d0de3a5a1/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
raw_datasets_google

DatasetDict({
    train: Dataset({
        features: ['en', 'th'],
        num_rows: 149411
    })
})

In [ ]:
# check for invalid value for en (open sub title)
from tqdm.notebook import tqdm
for index,text in tqdm(enumerate(raw_datasets_google['train']['en'])):
  if not isinstance(text, str):
    print(index)

0it [00:00, ?it/s]

In [ ]:
for index,text in tqdm(enumerate(raw_datasets_google['train']['th'])):
  if not isinstance(text, str):
    print(index)

0it [00:00, ?it/s]

In [ ]:
# load clean dataset
raw_datasets_gnome = load_dataset("csv", data_files="/content/drive/MyDrive/NMT Hackathon/Dataset/TrainSet/Clean/GNOME_dataset.csv")
raw_datasets_LST = load_dataset("csv", data_files="/content/drive/MyDrive/NMT Hackathon/Dataset/TrainSet/Clean/LST_Tokenize_Upper_Cleaned_reform.csv")
raw_datasets_QED = load_dataset("csv", data_files="/content/drive/MyDrive/NMT Hackathon/Dataset/TrainSet/Clean/QED_Clean_reform.csv")
raw_datasets_Tanzil = load_dataset("csv", data_files="/content/drive/MyDrive/NMT Hackathon/Dataset/TrainSet/Clean/Tanzil_dataset_reform.csv")
raw_datasets_Ubuntu = load_dataset("csv", data_files="/content/drive/MyDrive/NMT Hackathon/Dataset/TrainSet/Clean/Ubuntu_dataset.csv")
raw_datasets_bible = load_dataset("csv", data_files="/content/drive/MyDrive/NMT Hackathon/Dataset/TrainSet/Clean/bible_dataset_reform.csv")
raw_datasets_jw300 = load_dataset("csv", data_files="/content/drive/MyDrive/NMT Hackathon/Dataset/TrainSet/Clean/jw300_reform.csv")
raw_dataset_sub = load_dataset("csv", data_files="/content/drive/MyDrive/NMT Hackathon/Dataset/TrainSet/Clean/sub_finallycleaned_reform_edit.csv")

In [ ]:
print(raw_datasets_gnome['train'][0])
print(raw_datasets_LST['train'][0])
print(raw_datasets_QED['train'][0])
print(raw_datasets_Tanzil['train'][0])
print(raw_datasets_Ubuntu['train'][0])
print(raw_datasets_bible['train'][0])
print(raw_datasets_jw300['train'][0])
print(raw_dataset_sub['train'][0])

{'en': 'Give your application an accessibility workout', 'th': 'ช่วยพัฒนาสิ่งอำนวยความสะดวกในโปรแกรมของคุณ'}
{'en': "Thank you but I don't have enough time", 'th': 'ขอบคุณแต่ฉันไม่มีเวลาพอ'}
{'en': 'This is the same problem that we had in the last video.', 'th': 'นี่คือปัญหาข้อเดิมที่เราทำในวิดีโอที่แล้ว'}
{'en': 'Surefy we would have been the bondmen of God sincere.', 'th': '“แน่นอน เราก็จะเป็นบ่าวของอัลลอฮฺผู้ซื่อสัตย์”'}
{'en': 'AisleRiot Solitaire', 'th': 'ถอดไพ่ AisleRiot'}
{'en': 'In the beginning God created the heavens and the earth.', 'th': 'ในเริ่มแรกนั้นพระเจ้าทรงเนรมิตสร้างฟ้าและแผ่นดินโลก'}
{'en': '“ Sins committed in the dark are seen in Heaven like sheets of fire . ”  — Chinese proverb', 'th': '“บาปซึ่งได้กระทำในที่มืดปรากฏดุจเปลวเพลิงในสวรรค์.”—สุภาษิตจีน'}
{'en': 'You can be champion.', 'th': 'ฉันว่านายเป็นแช็มป์แล้วนะ'}


In [ ]:
# check for invalid value for en (open sub title)
from tqdm.notebook import tqdm
for index,text in tqdm(enumerate(raw_dataset_sub['train']['en'])):
  if not isinstance(text, str):
    print(index)

0it [00:00, ?it/s]

In [ ]:
for index,text in tqdm(enumerate(raw_dataset_sub['train']['th'])):
  if not isinstance(text, str):
    print(index)

0it [00:00, ?it/s]

In [ ]:
#concat clean dataset
#####!!!!!! ubuntu ห้าม ใช้ !!!! ######
raw_datasets_cc_big = concatenate_datasets([raw_datasets_gnome['train'], raw_datasets_LST['train'] , raw_datasets_QED['train'] ,raw_datasets_Tanzil['train']  , raw_datasets_bible['train'] , raw_datasets_jw300['train'],raw_dataset_sub['train']])

In [ ]:
raw_datasets_cc_big

Dataset({
    features: ['en', 'th'],
    num_rows: 3539244
})

In [ ]:
# concat SCB + opus dataset
raw_datasets_cc_train = concatenate_datasets([raw_datasets_opus['train'], raw_datasets_scb['train']])
raw_datasets_cc_valid = concatenate_datasets([raw_datasets_opus['validation'], raw_datasets_scb['validation']])
raw_datasets_cc_test = concatenate_datasets([raw_datasets_opus['test'], raw_datasets_scb['test']])

In [ ]:
raw_datasets_cc = concatenate_datasets([raw_datasets_cc_train,raw_datasets_cc_valid,raw_datasets_cc_test])

In [ ]:
raw_datasets_cc =  raw_datasets_cc.remove_columns(['subdataset'])

In [ ]:
raw_datasets_cc_big

Dataset({
    features: ['en', 'th'],
    num_rows: 3539244
})

In [ ]:
raw_datasets_cc

Dataset({
    features: ['translation'],
    num_rows: 2005752
})

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "/content/drive/MyDrive/NMT Hackathon/model/Finetune_Final/checkpoint-346564"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")
tokenizer.source_lang = 'en'
tokenizer.target_lang = 'th'

In [ ]:
# test input,target sentence scb+opus

en_sentence = raw_datasets_cc[-100]["translation"]["en"]
th_sentence = raw_datasets_cc[-100]["translation"]["th"]

inputs = tokenizer(en_sentence)
with tokenizer.as_target_tokenizer():
    targets = tokenizer(th_sentence)

print(tokenizer.convert_ids_to_tokens(inputs["input_ids"]))
print(tokenizer.convert_ids_to_tokens(targets["input_ids"]))

['▁Oh', '▁weekend', '▁should', '▁be', '▁fine', '.', '▁Around', '▁2', 'pm', '?', '</s>']
['▁', 'โอ', '้', '▁', 'วัน', 'ห', 'ย', 'ุ', 'ด', 'น่า', 'จะ', 'ได้', 'นะ', '▁', 'ประ', 'มา', 'ณ', 'บ', '่า', 'ย', 'ส', 'อง', 'ได้', 'ไหม', '</s>']


In [ ]:
# test input,target sentence clean text

en_sentence = raw_datasets_cc_big[-1250]["en"]
th_sentence = raw_datasets_cc_big[-1250]["th"]

inputs = tokenizer(en_sentence)
with tokenizer.as_target_tokenizer():
    targets = tokenizer(th_sentence)

print(tokenizer.convert_ids_to_tokens(inputs["input_ids"]))
print(tokenizer.convert_ids_to_tokens(targets["input_ids"]))

['▁That', "'", 's', '▁all', '▁we', '▁know', '.', '</s>']
['▁', 'นั่น', 'คือ', 'ทั้ง', 'หม', 'ด', 'ที่', 'เรา', 'ท', 'รา', 'บ', '</s>']


In [ ]:
max_input_length = 128
max_target_length = 128


def preprocess_function_scb(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["th"] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# apply that preprocessing in one go on all the splits of our dataset:

tokenized_datasets = raw_datasets_cc.map(
    preprocess_function_scb,
    batched=True,
    remove_columns = raw_datasets_cc.column_names,
)

  0%|          | 0/2006 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2005752
})

In [ ]:
raw_datasets_google

DatasetDict({
    train: Dataset({
        features: ['en', 'th'],
        num_rows: 149411
    })
})

In [ ]:
max_input_length = 128
max_target_length =128
def preprocess_function_bigdata(examples):
    inputs = [ex for ex in examples["en"]]
    targets = [ex for ex in examples["th"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# apply that preprocessing in one go on all the splits of our dataset:

tokenized_datasets_2 = raw_datasets_cc_big.map(
    preprocess_function_bigdata,
    batched=True,
    remove_columns = raw_datasets_cc_big.column_names,
)

NameError: ignored

In [ ]:
raw_datasets_google

DatasetDict({
    train: Dataset({
        features: ['en', 'th'],
        num_rows: 149411
    })
})

In [ ]:
raw_datasets_google = concatenate_datasets([raw_datasets_google['train']])

In [ ]:
raw_datasets_google

Dataset({
    features: ['en', 'th'],
    num_rows: 149411
})

In [ ]:
tokenized_datasets_2 = raw_datasets_google.map(
    preprocess_function_bigdata,
    batched=True,
    remove_columns = raw_datasets_google.column_names,
)

  0%|          | 0/150 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets_2

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 149411
})

In [ ]:
raw_datasets_final = concatenate_datasets([tokenized_datasets,tokenized_datasets_2])

In [ ]:
raw_datasets_final

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5544996
})

In [ ]:
#raw_datasets_final['input_ids'][0]

In [ ]:
tokenized_datasets = raw_datasets_final.shuffle()

In [ ]:
tokenized_datasets = tokenized_datasets_2.shuffle()

In [ ]:
#tokenized_datasets['input_ids'][0]

# Training

In [ ]:
from transformers import AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
# Inspect with padding (must be -100 for label)
batch = data_collator([tokenized_datasets[i] for i in range(1, 3)])
batch.keys()


dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [ ]:
batch["labels"]

tensor([[    4,  1948,  3613,  2395, 16283, 10419,     4, 28411,  5438, 10552,
          3187,  1807,  5003,  9121, 27418, 22630,  2242,  3329, 15351,     4,
         24190, 20366,  3187,  3613,  2488,  3995,  2711,  3556, 12131, 18001,
         18224, 12324,  5676, 21990,  3329,  3571,  3556,  2242, 20175,  2939,
         12581,  7933,  9619,  3556, 12324,  2939,  3398,  2488, 27407, 13908,
          2939, 26936, 10649,  2242,  7933,  5003,  2488, 17568, 10649,  2670,
         24190,  2670,  3571,  3329, 11363,  5003, 16823,  3187,  2242,  3995,
          4896, 17055, 12324,  3187,  1807,  5003,  9121, 15232,  2711,  9891,
          3571, 15351,  3187,  5003,  2488,  5585,  8443,  2542, 15250,  9524,
          7284, 10595, 20175,  7933,  9619,  3556,  2670, 21990, 21900,  2488,
         23376,  1921,  5878, 16509,  3398,  3995, 24139,  5676,  5003,  2711,
         16509,  3398,  3995,     0],
        [    4,  5585, 16594, 30335,  5438, 27751, 10552,  7933, 21990,  9524,
         17222

In [ ]:
batch["decoder_input_ids"]

tensor([[64109,     4,  1948,  3613,  2395, 16283, 10419,     4, 28411,  5438,
         10552,  3187,  1807,  5003,  9121, 27418, 22630,  2242,  3329, 15351,
             4, 24190, 20366,  3187,  3613,  2488,  3995,  2711,  3556, 12131,
         18001, 18224, 12324,  5676, 21990,  3329,  3571,  3556,  2242, 20175,
          2939, 12581,  7933,  9619,  3556, 12324,  2939,  3398,  2488, 27407,
         13908,  2939, 26936, 10649,  2242,  7933,  5003,  2488, 17568, 10649,
          2670, 24190,  2670,  3571,  3329, 11363,  5003, 16823,  3187,  2242,
          3995,  4896, 17055, 12324,  3187,  1807,  5003,  9121, 15232,  2711,
          9891,  3571, 15351,  3187,  5003,  2488,  5585,  8443,  2542, 15250,
          9524,  7284, 10595, 20175,  7933,  9619,  3556,  2670, 21990, 21900,
          2488, 23376,  1921,  5878, 16509,  3398,  3995, 24139,  5676,  5003,
          2711, 16509,  3398,  3995],
        [64109,     4,  5585, 16594, 30335,  5438, 27751, 10552,  7933, 21990,
          9524

In [ ]:
# compare with actual label
for i in range(1, 3):
    print(tokenized_datasets[i]["labels"])

[4, 1948, 3613, 2395, 16283, 10419, 4, 28411, 5438, 10552, 3187, 1807, 5003, 9121, 27418, 22630, 2242, 3329, 15351, 4, 24190, 20366, 3187, 3613, 2488, 3995, 2711, 3556, 12131, 18001, 18224, 12324, 5676, 21990, 3329, 3571, 3556, 2242, 20175, 2939, 12581, 7933, 9619, 3556, 12324, 2939, 3398, 2488, 27407, 13908, 2939, 26936, 10649, 2242, 7933, 5003, 2488, 17568, 10649, 2670, 24190, 2670, 3571, 3329, 11363, 5003, 16823, 3187, 2242, 3995, 4896, 17055, 12324, 3187, 1807, 5003, 9121, 15232, 2711, 9891, 3571, 15351, 3187, 5003, 2488, 5585, 8443, 2542, 15250, 9524, 7284, 10595, 20175, 7933, 9619, 3556, 2670, 21990, 21900, 2488, 23376, 1921, 5878, 16509, 3398, 3995, 24139, 5676, 5003, 2711, 16509, 3398, 3995, 0]
[4, 5585, 16594, 30335, 5438, 27751, 10552, 7933, 21990, 9524, 17222, 12324, 26834, 9891, 3556, 3613, 1921, 9116, 2711, 16823, 31494, 20175, 9760, 2939, 5003, 3995, 3995, 2711, 17055, 26582, 5438, 15232, 3396, 2488, 4, 1807, 5878, 3571, 3187, 523, 17774, 26029, 3329, 3571, 3556, 2242, 39

In [ ]:
# setup metric
from datasets import load_metric
import numpy as np
metric = load_metric("sacrebleu")


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

Downloading:   0%|          | 0.00/2.36k [00:00<?, ?B/s]

In [ ]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    '/content/drive/MyDrive/NMT Hackathon/model/finetune_google',
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    save_total_limit=5,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets,
    #eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    #compute_metrics=compute_metrics,
)

Using amp half precision backend


In [ ]:
# check score before train
#trainer.evaluate(max_length=max_target_length)

In [ ]:
# Train!
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 149411
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 14010


Step,Training Loss
500,0.416000
1000,0.352900
1500,0.326400
2000,0.311900
2500,0.298300
3000,0.290600
3500,0.283300
4000,0.276200
4500,0.275200
5000,0.254300


Saving model checkpoint to /content/drive/MyDrive/NMT Hackathon/model/finetune_google/checkpoint-4670
Configuration saved in /content/drive/MyDrive/NMT Hackathon/model/finetune_google/checkpoint-4670/config.json
Model weights saved in /content/drive/MyDrive/NMT Hackathon/model/finetune_google/checkpoint-4670/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/NMT Hackathon/model/finetune_google/checkpoint-4670/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/NMT Hackathon/model/finetune_google/checkpoint-4670/special_tokens_map.json
Saving model checkpoint to /content/drive/MyDrive/NMT Hackathon/model/finetune_google/checkpoint-9340
Configuration saved in /content/drive/MyDrive/NMT Hackathon/model/finetune_google/checkpoint-9340/config.json
Model weights saved in /content/drive/MyDrive/NMT Hackathon/model/finetune_google/checkpoint-9340/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/NMT Hackathon/model/finetune_googl

TrainOutput(global_step=14010, training_loss=0.257622534355719, metrics={'train_runtime': 1801.4806, 'train_samples_per_second': 248.814, 'train_steps_per_second': 7.777, 'total_flos': 1.1867496586149888e+16, 'train_loss': 0.257622534355719, 'epoch': 3.0})

In [ ]:
#trainer.evaluate(max_length=max_target_length)